In [ ]:
import json


file1_path = '/content/train.jsonl' 
file2_path = '/content/eval_results_merged_qwen_14b.jsonl'
output_path = 'proc_eval_results_merged_qwen_14b.jsonl'

id_to_info_map = {}

print(f"Шаг 1: Чтение данных из {file1_path}...")

try:
    with open(file1_path, 'r', encoding='utf-8') as f1:
        for line in f1:
            data = json.loads(line)
            
            record_id = data['meta']['id']
            subject = data['inputs']['subject']
            domain = data['meta']['domain']
            
            id_to_info_map[record_id] = {'subject': subject, 'domain': domain}
            
    print(f"Успешно загружено {len(id_to_info_map)} записей с метаданными.")

except FileNotFoundError:
    print(f"Ошибка: Файл {file1_path} не найден.")
    exit()
except (KeyError, json.JSONDecodeError) as e:
    print(f"Ошибка при чтении {file1_path}. Проверьте структуру JSON. Ошибка: {e}")
    exit()



print(f"\nШаг 2: Обработка {file2_path} и запись в {output_path}...")

lines_processed = 0
lines_written = 0

try:
    with open(file2_path, 'r', encoding='utf-8') as f2, \
         open(output_path, 'w', encoding='utf-8') as fout:
        
        for line in f2:
            lines_processed += 1
            data_to_update = json.loads(line)
            
            record_id = data_to_update['id']
            
            if record_id in id_to_info_map:
                info_to_add = id_to_info_map[record_id]
                
                data_to_update['subject'] = info_to_add['subject']
                data_to_update['domain'] = info_to_add['domain']
                
                fout.write(json.dumps(data_to_update, ensure_ascii=False) + '\n')
                lines_written += 1
            else:
                print(f"ВНИМАНИЕ: ID {record_id} из файла {file2_path} не найден в {file1_path}. Строка пропущена.")

    print(f"\nГотово! Обработано строк: {lines_processed}. Записано строк: {lines_written}.")
    print(f"Результат сохранен в файл: {output_path}")

except FileNotFoundError:
    print(f"Ошибка: Файл {file2_path} не найден.")
except (KeyError, json.JSONDecodeError) as e:
    print(f"Ошибка при чтении {file2_path}. Проверьте структуру JSON. Ошибка: {e}")